In [1]:
import xarray as xr
import xesmf as xe
import os

In [2]:
os.chdir('/path/to/directory/clean')

Currently, all datasets are at a different spatial resolution (0.1 degree by 0.1 degree for IMERG and WWLLN, 0.25 by 0.25 degree for CAPE, and 0.5 by 0.625 degree for MERRA). All datasets will be regridded to the MERRA resolution for the time being. Additionally, the WWLLN dataset is at a 3-hourly temporal resolution, while the CAPE and MERRA datasets are hourly and the IMERG dataset is at a 30 minute temporal resolution. All will be resampled to 3-hourly.

In [3]:
merra = xr.open_dataset('MERRA_clean.nc')
imerg = xr.open_dataset('IMERG_clean.nc')
cape = xr.open_dataset('CAPE_clean.nc')
wwlln = xr.open_dataset('WWLLN_clean.nc')

In [4]:
merra = merra.resample(time='3h').mean()

In [5]:
target_grid = merra[['lat', 'lon']]
regridder = xe.Regridder(imerg, target_grid, 'conservative')
imerg_merra = regridder(imerg)
imerg_merra = imerg_merra.resample(time='3h').mean()
imerg_merra

/home/disk/p/rjones98/anaconda3/envs/plotenv/lib/python3.9/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


<xarray.Dataset> Size: 57MB
Dimensions:        (time: 2920, lat: 52, lon: 94)
Coordinates:
  * lat            (lat) float64 416B 24.5 25.0 25.5 26.0 ... 49.0 49.5 50.0
  * lon            (lon) float64 752B -125.0 -124.4 -123.8 ... -67.5 -66.88
  * time           (time) datetime64[ns] 23kB 2023-01-01 ... 2023-12-31T21:00:00
Data variables:
    precipitation  (time, lat, lon) float32 57MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    regrid_method:  conservative

In [6]:
cape = cape.rename({'latitude': 'lat', 'longitude': 'lon'})
target_grid = merra[['lat', 'lon']]
regridder = xe.Regridder(cape, target_grid, 'conservative')
cape_merra = regridder(cape)
cape_merra = cape_merra.resample(time='3h').mean()
cape_merra

<xarray.Dataset> Size: 114MB
Dimensions:  (time: 2920, lat: 52, lon: 94)
Coordinates:
  * lat      (lat) float64 416B 24.5 25.0 25.5 26.0 26.5 ... 48.5 49.0 49.5 50.0
  * lon      (lon) float64 752B -125.0 -124.4 -123.8 ... -68.12 -67.5 -66.88
  * time     (time) datetime64[ns] 23kB 2023-01-01 ... 2023-12-31T21:00:00
Data variables:
    cape     (time, lat, lon) float64 114MB 9.55e-13 0.1126 ... 1.365e-12
Attributes:
    regrid_method:  conservative

In [7]:
wwlln = wwlln.sel(lat=slice(24.5, 50), lon=slice(-125, -66.5))

target_grid = merra[['lat', 'lon']]
regridder = xe.Regridder(wwlln, target_grid, 'conservative')
wwlln_merra = regridder(wwlln)
wwlln_merra

<xarray.Dataset> Size: 228MB
Dimensions:  (time: 2920, lat: 52, lon: 94)
Coordinates:
  * time     (time) datetime64[ns] 23kB 2023-01-01 ... 2023-12-31T21:00:00
  * lat      (lat) float64 416B 24.5 25.0 25.5 26.0 26.5 ... 48.5 49.0 49.5 50.0
  * lon      (lon) float64 752B -125.0 -124.4 -123.8 ... -68.12 -67.5 -66.88
Data variables:
    ltg      (time, lat, lon) float64 114MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    energy   (time, lat, lon) float64 114MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    regrid_method:  conservative

In [9]:
ds = xr.merge([cape_merra, imerg_merra, merra, wwlln_merra])
ds
ds.to_netcdf('/path/to/directory/ai_ready/AI_ready_dataset.nc')